## Location Data - Partial
I was able to collect a list of 'location based' subreddits, and associate as many as I could with their actual, USA recognized counties using a geofencing API and the information provided by a reddit post summarizing all of the location subreddits by US state. This is just a partial data set, though, because I hit an error in the middle and lost my connection. Didn't have enough error handling implemented to gracefully take the error. But this should be enough data to atleast start writing the scripts for the rest of the data collection. Once that is proven to be working/interesting I'll go back and udpate the original data gather scripts to ensure they'll query the entire set of location subreddits. 

To start, I just want to open up the csv, get some stats, and start looking at a few individual subreddits to see if i can pull some users. I'll have to use PRAW again, so might need to muck about with the config/plumbing.

In [3]:
import pandas as pd
import praw
from psaw import PushshiftAPI
import time
import datetime

reddit = praw.Reddit("data_enrich", user_agent="data_project_ua")
psapi  = PushshiftAPI(reddit)

FN = "../data/location_subreddit_counties.csv"

location_subs = pd.read_csv(FN)

In [4]:
# lets get some users from a sub
sub_url = location_subs.iloc[0][2] # gets the /r/NNNN for the subreddit
sub = reddit.subreddit(sub_url)

print(sub)

/r/alabama


Just found some helpful comments on github:

    Use Subredit.submissions(start=None, end=None, extra_query=None). start and end are unix timestamps and extra_query is an additional search term for narrowing results further.

    For your use-case, your third line should be:

    for submission in reddit.subreddit(subreddit).submissions(start=1456790400, end=1459468800, extra_query=search_lib):

    This will yield submissions, newest first, in between the dates of March 1 and April 1. You can adjust the timestamps based on the dates you wish to search.

In [5]:
TARGET_YEAR = 2016

START_TS = int(time.mktime(datetime.date(TARGET_YEAR, 1,   1).timetuple()))
END_TS   = int(time.mktime(datetime.date(TARGET_YEAR, 12, 30).timetuple()))

Great looks like this feature was deprecated. Apparently the answer is 'use pushshift' but pushshift is down or something so fuck me right?

I think I have its wrapper install and working. Here's a snippet from the docs that should get me going:

````
import datetime as dt
start_epoch=int(dt.datetime(2017, 1, 1).timestamp())
list(api.search_submissions(after=start_epoch,
                            subreddit='politics',
                            filter=['url','author', 'title', 'subreddit'],
                            limit=10))
````

In [6]:
results = list(psapi.search_submissions(after=START_TS,
                              before=END_TS,
                              subreddit='politics',
                              filter=['url','author', 'title', 'subreddit'],
                              limit=10))

Ok so that works? Holy shit? So we can easily do this for a time frame, and limit to a single, specific subreddit while we do so.

In [ ]:
for r in results:
    print(r.title, r.url)

I need to make sure that I can use a time stamp when searching a users past shit. 

In [18]:
results = list(psapi.search_comments(after=START_TS,
                                     before=END_TS,
                                     subreddit='RimWorld',
#                                      author='willpower12',
                                     limit=100))

In [19]:
for r in results:
    print(r.subreddit, r.body)

RimWorld This game has never been on sale and probably won't be anytime soon.
RimWorld Ham? I thought it tasted like veal.
RimWorld There's no downsides? I've just assumed my colonists wouldn't like it.
RimWorld I have no audio related mods, and as I said it happens even with all my mods disabled.

But it doesn't hurt to post so here anyway:

Core

HugsLib

RomanceDiversified

Prepare for Romance! (edb's prepare carefully with a fourth trait slot)

Moody

MineItAll

LongRangePodLauncher

Advanced Hydroponics Basin

Outfitter A16

20th Century Weapons Mod (20CWM)

Psychology

ED-Laser Drill

FashionRIMsta

Rumors and Deception

A Dog Said...

Realistic Darkness Lighter Version V2.0 A16

ProgrammableVents_A16

Rimsenal - Rimhair

Remote Explosives

Nackblad Inc Rimhair

A World Without Hat

I Can Fix It!

Look At Me, I'm The Worker Now

More Trade Ships

ED-Embrasures

RT Fuse

[T] MoreFloors

Efficient Light

Colony Manager

Medical Tab

WorkTab

Relations Tab

Fluffy Breakdowns

Bluepr

Oh look at that we can! Yay! So theres not gotchas with the APIs left to cover. Need to automate getting the db up. 

EVEN BETTER! We can limit by subreddit. 